In [2]:
import numpy as np
import pandas as pd
import math
from scipy.stats import poisson
import time
import matplotlib.pyplot as plt
import plotly.express as px


## Functions

In [3]:
I = 48  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

print(N)

100


In [4]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [5]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [6]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probS = calcExponentialLimit(beta*N)+1
print(probS)
calculate_p(beta, probS, precision)


1021


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [7]:
def calculate_v(p, beta, precision, precision_limit, N, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*N) + 1
  v = np.zeros((N+1, limit+d))
  add_v = np.zeros((N+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, N+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(N+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probS, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.62654756e-08, 2.23475899e-08, 1.02085262e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 1020)

In [8]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########

v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)


In [9]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, N, no_show, beta):
  return (I * d) + tardiness - (N * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, N):
  w = np.zeros((I+1, N+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= N
  return waitingtime

##########
## TEST ##
##########

calcWaitingtime(p_min, x, p, limit, I, N)


331.53348741763307

In [10]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess


##########
## TEST ##
##########

# calcFracExcess(p_min, I)


<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [11]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nN: {sum(x)},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probS, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [2 1 2 2 2 1 2 4 4 3 3 4 3 1 1 0 4 0 1 4 3 4 1 4 2 1 3 0 0 2 4 3 2 2 1 1 1
 2 2 3 2 2 4 1 0 3 0 3],
N: 100,
Objective value: 393.9178421766069,
Prob calculation time: 19.384422 sec,
Waiting time (timer): 331.53348741763307 (43.613004 sec),
Idle time (timer): -4.492587984077318 (0.000003 sec),
Tardiness (timer): 655.5074120159227 (0.000364 sec)



{'p_min': array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [3.23993451e-004, 7.19452059e-004, 1.85001958e-003, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [1.54361169e-004, 2.59311131e-004, 5.87758961e-004, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        ...,
        [3.05053708e-150, 1.51679442e-149, 7.90108571e-149, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [9.53580096e-147, 3.06407291e-146, 1.24658458e-145, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [5.43832095e-153, 2.65782282e-152, 1.35986751e-151, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000]]),
 'waitingTime': 331.53348741763307,
 'idleTime': -4.492587984077318,
 'tardiness': 655.5074120159227,
 'objVal': 393.9178421766069}

In [12]:
def createDist(arr, reverse=False):
    minX, maxX = arr.min(), arr.max()
    minY, maxY = 0, 1
    if(reverse):
        minY, maxY = maxY, minY
    # Scale to values between 0, 1 and make total sum equal to 1
    arrScaled = np.interp(arr, (minX, maxX), (minY, maxY))
    arrNorm = [x / arrScaled.sum() for x in arrScaled]
    return(arrNorm)
    

##########
## TEST ##
##########

arr = np.random.choice(10, 5)
dist1 = createDist(arr)
dist2 = createDist(arr, reverse=True)
arr, dist1, sum(dist1), dist2, sum(dist2)


(array([5, 5, 5, 4, 3]),
 [0.2857142857142857,
  0.2857142857142857,
  0.2857142857142857,
  0.14285714285714285,
  0.0],
 1.0,
 [0.0, 0.0, 0.0, 0.3333333333333333, 0.6666666666666666],
 1.0)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [13]:
## OLD
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    # norm1 = sum(child1)
    # p1 = child1 / norm1
    p1 = createDist(child1)
    
    # The number of swaps should not exceed the maximum number of patients in any interval. Otherwise it might happen that the number of patients that are transfered exceeds availability, resulting in a negative amount.
    max1 = int(max(child1))
    max2 = int(max(child2))
    nSwaps = min(nSwaps, max1, max2)
    
    # Swap patients parent1 to parent 2
    i = np.random.choice(I, nSwaps, p=p1)
    for i in i:
        child1[i] += -1
        child2[i] += 1
    
    # Swap patients parent2 to parent 1
    # norm2 = sum(child2)
    # p2 = child2 / norm2
    p2 = createDist(child2)
    
    j = np.random.choice(I, nSwaps, p=p2)
    for j in j:
        child1[j] += 1
        child2[j] += -1
    return(child1, child2)

##########
## TEST ##
##########


parent1 = np.random.choice(5, I)
parent2 = np.random.choice(5, I)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[0 2 3 4 1 0 4 4 0 3 4 0 4 1 2 0 4 3 0 4 0 2 2 2 2 4 1 4 2 4 2 0 1 2 4 0 2
 4 0 2 1 3 2 0 3 1 4 3] [4 4 4 2 2 0 3 0 2 2 0 3 3 1 2 4 2 1 0 1 0 2 1 1 3 2 1 0 0 2 3 3 1 4 1 0 4
 4 3 2 0 0 4 1 2 4 3 2]


(array([0, 3, 3, 4, 1, 0, 4, 2, 0, 4, 4, 0, 4, 1, 2, 0, 4, 3, 0, 4, 0, 2,
        2, 2, 2, 3, 1, 4, 2, 5, 2, 0, 1, 2, 4, 0, 2, 4, 0, 2, 1, 3, 2, 0,
        3, 1, 4, 3]),
 array([4, 3, 4, 2, 2, 0, 3, 2, 2, 1, 0, 3, 3, 1, 2, 4, 2, 1, 0, 1, 0, 2,
        1, 1, 3, 3, 1, 0, 0, 1, 3, 3, 1, 4, 1, 0, 4, 4, 3, 2, 0, 0, 4, 1,
        2, 4, 3, 2]))

In [14]:
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        child1[i] += -1
        child2[i] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        child2[i] += -1
        child1[i] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[0 0 0 1 3 3 0 4 2 4] [2 2 0 3 0 1 4 3 4 2]


(array([1, 1, 0, 1, 2, 2, 0, 4, 2, 4]), array([1, 1, 0, 3, 1, 2, 4, 3, 4, 2]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [15]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[1 1 0 1 2 2 0 4 2 4]
[4 1 0 1 2 2 0 4 2 1]


## Classes

#### Suggesties: [get / set methods](https://www.geeksforgeeks.org/getter-and-setter-in-python/) voor parameters / met tests / nuttig voor optimaliseren hyper parameters, bv mbv bayesian stats

Create a data object to store and generate initial data.

In [16]:
class Data:
    def __init__(self, I=10, N=12, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5, nSwaps=4, mutationRate=0.1):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
        self.nSwaps, self.mutationRate = nSwaps, mutationRate
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W, self.nSwaps, self.mutationRate),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W', 'nSwaps', 'mutationRate'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8', 'i4', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]


array((100, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 60})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [17]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)


##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
schedule = scheduleObject.get_schedule()
schedule

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]


array([3, 4, 0, 3, 2, 3, 2, 2, 2, 3, 2, 4, 2, 2, 3, 2, 3, 4, 1, 1, 3, 2,
       3, 0, 0, 1, 1, 1, 2, 1, 3, 3, 0, 1, 0, 1, 3, 1, 3, 4, 3, 1, 2, 2,
       3, 0, 5, 3])

Create population of individuals

In [18]:
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in self.individuals])
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show,
                                            I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in newIndividuals])
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


## Tests

In [19]:
##########
## TEST ##
##########

bestRank = []
dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [3 2 0 2 0 1 3 0 1 0],
N: 12,
Objective value: 33.98508955321972,
Prob calculation time: 0.091673 sec,
Waiting time (timer): 33.22834285218503 (0.124531 sec),
Idle time (timer): -0.05816374574558836 (0.000002 sec),
Tardiness (timer): 57.94183625425442 (0.000054 sec)

Schedule: [1 1 0 1 0 3 0 1 3 2],
N: 12,
Objective value: 29.121909760115923,
Prob calculation time: 0.080410 sec,
Waiting time (timer): 22.18934656473735 (0.080575 sec),
Idle time (timer): 1.2544729554944922 (0.000002 sec),
Tardiness (timer): 59.2544729554945 (0.000054 sec)

Schedule: [0 0 1 1 1 1 3 3 1 1],
N: 12,
Objective value: 37.61841032376368,
Prob calculation time: 0.079185 sec,
Waiting time (timer): 30.402366777489863 (0.063688 sec),
Idle time (timer): 10.034453870037495 (0.000001 sec),
Tardiness (timer): 68.0344538700375 (0.000051 sec)

Schedule: [0 0 3 1 1 0 4 1 1 1],
N: 12,
Objective value: 39.39737161194953,
Prob calculation time: 0.083314 sec,
Waiting time (timer): 34.0609065933614 (0.069951 sec),
Id

In [20]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([3, 2, 0, 2, 0, 1, 3, 0, 1, 0]), array([1, 1, 0, 1, 0, 3, 0, 1, 3, 2]), array([0, 0, 1, 1, 1, 1, 3, 3, 1, 1]), array([0, 0, 3, 1, 1, 0, 4, 1, 1, 1]), array([2, 1, 0, 0, 0, 3, 1, 3, 1, 1])] 
 [array([3, 2, 0, 2, 0, 1, 3, 0, 1, 0]), array([1, 1, 0, 1, 0, 3, 0, 1, 3, 2]), array([0, 0, 1, 1, 1, 1, 3, 3, 1, 1]), array([0, 0, 3, 1, 1, 0, 4, 1, 1, 1]), array([2, 1, 0, 0, 0, 3, 1, 3, 1, 1])]


In [21]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [3 2 0 2 0 1 3 0 1 0],
N: 12,
Objective value: 33.98508955321972,
Prob calculation time: 0.091916 sec,
Waiting time (timer): 33.22834285218503 (0.107878 sec),
Idle time (timer): -0.05816374574558836 (0.000002 sec),
Tardiness (timer): 57.94183625425442 (0.000064 sec)

Schedule: [1 1 0 1 0 3 0 1 3 2],
N: 12,
Objective value: 29.121909760115923,
Prob calculation time: 0.078712 sec,
Waiting time (timer): 22.18934656473735 (0.083918 sec),
Idle time (timer): 1.2544729554944922 (0.000002 sec),
Tardiness (timer): 59.2544729554945 (0.000052 sec)

Schedule: [0 0 1 1 1 1 3 3 1 1],
N: 12,
Objective value: 37.61841032376368,
Prob calculation time: 0.087658 sec,
Waiting time (timer): 30.402366777489863 (0.063555 sec),
Idle time (timer): 10.034453870037495 (0.000002 sec),
Tardiness (timer): 68.0344538700375 (0.000062 sec)

Schedule: [0 0 3 1 1 0 4 1 1 1],
N: 12,
Objective value: 39.39737161194953,
Prob calculation time: 0.081617 sec,
Waiting time (timer): 34.0609065933614 (0.070601 sec),
Id

[array([1, 1, 0, 1, 1, 0, 3, 3, 1, 1]),
 array([2, 0, 0, 1, 0, 3, 2, 0, 1, 3]),
 array([1, 1, 0, 2, 0, 3, 1, 1, 1, 2]),
 array([1, 1, 0, 2, 0, 2, 3, 1, 1, 1]),
 array([1, 2, 0, 0, 0, 3, 1, 2, 3, 0])]

In [22]:
# Initial population
bestRank = []
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    print(f'Generation: {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals


Schedule: [2 0 3 1 2 0 1 1 1 1],
N: 12,
Objective value: 32.34367563311346,
Prob calculation time: 0.084394 sec,
Waiting time (timer): 29.918730278015204 (0.071084 sec),
Idle time (timer): -0.031379011788274624 (0.000002 sec),
Tardiness (timer): 57.968620988211725 (0.000054 sec)

Schedule: [1 2 0 0 1 1 2 3 1 1],
N: 12,
Objective value: 29.621303904891818,
Prob calculation time: 0.083518 sec,
Waiting time (timer): 24.259198191729297 (0.072383 sec),
Idle time (timer): 0.1834096180543412 (0.000002 sec),
Tardiness (timer): 58.18340961805434 (0.000052 sec)

Schedule: [1 1 2 1 0 2 2 0 2 1],
N: 12,
Objective value: 30.745904075445274,
Prob calculation time: 0.080311 sec,
Waiting time (timer): 26.65246274770533 (0.084844 sec),
Idle time (timer): 0.03934540318522295 (0.000002 sec),
Tardiness (timer): 58.03934540318522 (0.000051 sec)

Schedule: [1 1 2 2 1 0 0 1 3 1],
N: 12,
Objective value: 30.745725257359616,
Prob calculation time: 0.079971 sec,
Waiting time (timer): 26.652448137355318 (0.07155

In [23]:

bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF


,Score,Schedule
0,29.290655,"[1, 2, 0, 2, 0, 0, 1, 2, 2, 2]"
1,28.321203,"[1, 1, 0, 1, 0, 1, 1, 2, 2, 3]"
2,27.905705,"[1, 1, 0, 1, 0, 1, 1, 1, 2, 4]"


In [24]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 1 0 0 1 0 1 0 1 2 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1
 3 0 1 1 0 0 0 0 0 0 0],
N: 24,
Objective value: 13.171321983750392,
Prob calculation time: 1.371818 sec,
Waiting time (timer): 11.837597579350243 (0.172271 sec),
Idle time (timer): 30.060471586683832 (0.000002 sec),
Tardiness (timer): 6.060471586683819 (0.000095 sec)

Schedule: [1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 3 0 0 1 1 0 0 1 1 0 3 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 2 1 2 1 0 0],
N: 24,
Objective value: 26.957709186730796,
Prob calculation time: 1.573652 sec,
Waiting time (timer): 12.924615756584972 (0.361939 sec),
Idle time (timer): 52.313104806827994 (0.000002 sec),
Tardiness (timer): 28.31310480682801 (0.000096 sec)

Schedule: [0 1 0 2 0 0 3 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 2 1 0 1 0 0 0 1 2 0 1 0 0 2 1
 0 1 0 1 0 1 0 0 0 0 0],
N: 24,
Objective value: 11.73320896614483,
Prob calculation time: 1.405818 sec,
Waiting time (timer): 12.009967425345414 (0.380604 sec),
Idle time (timer): 27.54870332667778 (0.000

In [25]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',hover_data=['Schedule'])
fig.update_traces(line_color='LimeGreen')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [26]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 10

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 3 1 0 0 0 0 0 0 0 2 0 1 2 1 1 1 1 1 1
 1 1 0 1 0 0 0 1 0 0 1],
N: 24,
Objective value: 38.21757363768787,
Prob calculation time: 1.400074 sec,
Waiting time (timer): 23.740456379221524 (0.240275 sec),
Idle time (timer): 63.86898514333211 (0.000002 sec),
Tardiness (timer): 39.86898514333208 (0.000097 sec)

Schedule: [1 1 0 0 1 0 1 2 1 1 0 0 0 1 0 0 0 0 2 0 1 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0
 1 0 2 0 1 0 0 1 0 1 0],
N: 24,
Objective value: 10.081309797057319,
Prob calculation time: 1.410758 sec,
Waiting time (timer): 6.859560767661495 (0.207131 sec),
Idle time (timer): 28.229142483321198 (0.000002 sec),
Tardiness (timer): 4.229142483321201 (0.000101 sec)

Schedule: [0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 4 0 0 1 1 1 0 0 1 0 1 1 1 2 0 0 0 0 0 1
 0 0 1 0 1 1 0 0 1 0 1],
N: 24,
Objective value: 22.194066795672825,
Prob calculation time: 1.446120 sec,
Waiting time (timer): 17.607042036636148 (0.224726 sec),
Idle time (timer): 41.25208330169727 (0.00000

In [27]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'])
fig.update_traces(line_color='RoyalBlue')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [28]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 20

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [1 1 1 2 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 2 0 0 3 0 0 0 0 0 2 0 0 1 0 0
 0 0 0 0 0 1 1 0 1 0 0],
N: 24,
Objective value: 11.97698764529582,
Prob calculation time: 1.435709 sec,
Waiting time (timer): 16.153004653082515 (0.323992 sec),
Idle time (timer): 25.192976306771357 (0.000002 sec),
Tardiness (timer): 1.1929763067713526 (0.000103 sec)

Schedule: [1 0 0 0 2 0 0 0 0 1 0 0 0 3 0 1 0 0 2 0 0 0 1 0 0 1 0 0 0 2 0 0 1 2 0 2 0
 0 0 0 1 0 1 1 0 1 1 0],
N: 24,
Objective value: 17.914490078702695,
Prob calculation time: 1.527388 sec,
Waiting time (timer): 10.020708015848475 (0.468381 sec),
Idle time (timer): 39.17701145393883 (0.000002 sec),
Tardiness (timer): 15.177011453938842 (0.000101 sec)

Schedule: [0 3 0 0 0 1 0 0 0 0 0 3 0 0 0 2 0 0 1 0 0 3 0 0 0 1 0 0 3 0 0 2 0 1 0 1 0
 0 0 1 1 0 0 0 0 1 0 0],
N: 24,
Objective value: 12.602848030953396,
Prob calculation time: 1.519262 sec,
Waiting time (timer): 14.219153444600797 (0.582348 sec),
Idle time (timer): 27.52531108852179 (0.00

In [29]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'])
fig.update_traces(line_color='Tomato')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [30]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 2 0 0 0 1 1 1 0 2 1 2 0 0 1 1 0 0 0 0 0
 0 1 0 1 0 2 0 2 2 0 0],
N: 24,
Objective value: 40.6058158296773,
Prob calculation time: 1.424361 sec,
Waiting time (timer): 26.01666226826372 (0.516281 sec),
Idle time (timer): 66.33191820395302 (0.000002 sec),
Tardiness (timer): 42.33191820395301 (0.000097 sec)

Schedule: [1 0 0 0 0 1 0 1 1 0 2 0 0 1 0 0 2 0 0 0 0 0 1 0 0 0 0 0 2 0 1 0 0 1 1 2 0
 0 0 0 0 2 0 0 1 0 3 1],
N: 24,
Objective value: 26.826812837859592,
Prob calculation time: 1.416260 sec,
Waiting time (timer): 8.128986504783276 (0.443135 sec),
Idle time (timer): 55.292030393243806 (0.000002 sec),
Tardiness (timer): 31.29203039324379 (0.000097 sec)

Schedule: [0 0 0 1 0 0 1 1 1 3 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 2
 0 0 0 1 0 0 4 1 1 1 0],
N: 24,
Objective value: 34.28804392447779,
Prob calculation time: 1.412334 sec,
Waiting time (timer): 15.398610964467357 (0.310241 sec),
Idle time (timer): 62.88099923115141 (0.000002 s

In [31]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'])
fig.update_traces(line_color='DarkOrange')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [32]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 1 1 1 0 0 0 1 0 1 0 1 1 2 1 0 0 0 0 0 1 0 2 0 0 1 1 0 0 1 0 2 1 0 0 1 1
 0 0 0 0 1 1 0 1 0 0 0],
N: 24,
Objective value: 9.21061678691825,
Prob calculation time: 1.414538 sec,
Waiting time (timer): 8.501523722628177 (0.208229 sec),
Idle time (timer): 25.683345496444957 (0.000002 sec),
Tardiness (timer): 1.6833454964449677 (0.000095 sec)

Schedule: [0 0 0 1 0 0 0 0 2 0 1 1 0 1 0 1 0 0 1 0 0 1 0 0 0 2 0 0 0 2 1 0 2 1 1 0 0
 1 0 0 0 0 0 1 0 3 1 0],
N: 24,
Objective value: 25.07359385279151,
Prob calculation time: 1.434214 sec,
Waiting time (timer): 11.740601551766687 (0.376875 sec),
Idle time (timer): 49.96225538680807 (0.000002 sec),
Tardiness (timer): 25.962255386808046 (0.000102 sec)

Schedule: [0 0 0 0 0 3 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 3 1 0
 2 0 2 0 0 0 0 0 1 0 2],
N: 24,
Objective value: 33.56726593651493,
Prob calculation time: 1.411394 sec,
Waiting time (timer): 16.11289122413069 (0.417869 sec),
Idle time (timer): 61.203515744771096 (0.000002

In [33]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'])
fig.update_traces(line_color='DarkCyan')
fig.show()
{fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/fileName}', include_plotlyjs="cdn")
}